In [1]:
import patsy
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
from scipy.signal import find_peaks
from scipy.optimize import minimize
from scipy.integrate import quad
from findpeaks import findpeaks
from quadprog import solve_qp

In [2]:
from module.auto_peak_valley import Peak_Valley_Simu, IsotonicReg, constrained_splines_reg, comprehensive_csr
from module.knot_func import *

In [3]:
def f(x):
    return np.sin(4*np.pi*x)

## 本研究方法

In [4]:
def to_softmax_v2(pv_coordinate: list): # 搞錯的 multi-class logistic reg 作法
    pv_coordinate_copy = np.array(pv_coordinate)
    output = pv_coordinate_copy / sum(pv_coordinate_copy)
    return output # array

def reverse_softmax_v2(pv_coordinate_softmax_form, x): # 搞錯的 multi-class logistic reg 作法
    pv_coordinate_original_form = 0.12012*(1/pv_coordinate_softmax_form[0])*pv_coordinate_softmax_form
    nearest_x = [min(x, key=lambda k: abs(k-j)) for j in pv_coordinate_original_form]
    nearest_x_index = [list(x).index(i) for i in nearest_x]
    nearest_x_index = [0] + nearest_x_index + [999]
    return nearest_x_index

def to_softmax_v3(pv_coordinate, x, base=0): # 最終版的 multi-class logistic reg 作法
    pv_coordinate_copy = np.hstack((x[0], np.array(pv_coordinate), x[-1]))
    pv_coordinate_copy2 = np.diff(pv_coordinate_copy)
    log_odds = np.log(pv_coordinate_copy2 / pv_coordinate_copy2[base])
    return log_odds

def reverse_softmax_v3(pv_coordinate_softmax_form, x): # 最終版的 multi-class logistic reg 作法
    # pv_coordinate_original_form = np.exp(pv_coordinate_softmax_form) * 0.24024
    # pv_coordinate_original_form = pv_coordinate_original_form.cumsum()[:-1]
    pv_coordinate_base_form = 1 / np.sum(np.exp(pv_coordinate_softmax_form))
    pv_coordinate_original_form = np.hstack((pv_coordinate_base_form, pv_coordinate_base_form*np.exp(pv_coordinate_softmax_form[1:]))).cumsum()
    pv_coordinate_original_form = pv_coordinate_original_form[:-1]
    nearest_x = [min(x, key=lambda k: abs(k-j)) for j in pv_coordinate_original_form]
    nearest_x_index = [list(x).index(i) for i in nearest_x]
    nearest_x_index = [0] + nearest_x_index + [999]
    return nearest_x_index

In [5]:
def coef1_list_func(n=200):
    coef1_list, knots_list = [], []
    for i in range(n):
        np.random.seed(i)
        x = np.linspace(0, 1, 1000).round(5)
        y = f(x) + np.random.normal(scale=0.3, size=(1000,))
        multi_peak = Peak_Valley_Simu()
        multi_peak.x, multi_peak.y = x, y
        multi_peak.auto_peak_points_detection_v3(step=0.02, distance=0.1, iter_scale=1)
        multi_peak.auto_valley_points_detection(step=0.02, distance=0.1, iter_scale=1)
        pv, pv_index = multi_peak.peak_valley_index()
        pv_coordinate_softmax_transform = to_softmax_v3(pv, x)
        minimize_result = minimize(multi_peak.isotonic_reg_rss_v3, pv_coordinate_softmax_transform, method="Nelder-Mead", options={"adaptive": True}).x
        new_pv = x[reverse_softmax_v3(minimize_result, x)]
        new_pv_index = reverse_softmax_v3(minimize_result, x)
        knots = get_knot(x, y)
        compre_csr = comprehensive_csr(x=x, y=y, pv_coordinate=new_pv[1:-1], pv_index=new_pv_index, knots_of_each_part=knots)
        try:
            coef1_list.append(compre_csr.Solve_QP(deg=3, meq=4))
        except ValueError:
            print("constraints are inconsistent, no solution")
            continue
        knots_list.append(compre_csr.comprehensive_knots())
    return knots_list, coef1_list

In [6]:
def ISE_list(knots, coef):
    ise_list = []
    for i, v in enumerate(knots):
        def fhat(w):
            bx = patsy.bs(x=w, knots=v, degree=3,
                        lower_bound=0, upper_bound=1,
                        include_intercept=True)
            ans = bx @ coef[i]
            return ans
        def SE(w):
            return (fhat(w) - f(w))**2
        ise_list.append(quad(SE, 0, 1)[0])
    return ise_list

In [ ]:
# knots_list, coef1_list = coef1_list_func()

In [8]:
# ise = ISE_list(knots_list, coef1_list)
# np.save(".\\weights\\thesis_ise_array", np.array(ise))

In [9]:
ise = np.load(".\\weights\\thesis_ise_array.npy")
min(ise)

0.0005066049690991673

## Scipy 方法

In [10]:
def coef2_list_func(n=200):
    coef2_list, knots_list = [], []
    for i in range(n):
        np.random.seed(i)
        x = np.linspace(0, 1, 1000).round(5)
        y = f(x) + np.random.normal(scale=0.3, size=(1000,))
        peaks, _ = find_peaks(y, distance=200, prominence=1, height=1)
        valleys, _ = find_peaks(-y, distance=200, prominence=1, height=1)
        pv_index2 = np.hstack((peaks, valleys))
        knots = get_knot(x, y)
        compre_csr = comprehensive_csr(x=x, y=y, pv_coordinate=x[pv_index2], pv_index=pv_index2, knots_of_each_part=knots)
        try:
            coef2_list.append(compre_csr.Solve_QP(deg=3, meq=4))
        except ValueError:
            print("constraints are inconsistent, no solution")
            continue
        knots_list.append(compre_csr.comprehensive_knots())
    return knots_list, coef2_list

In [11]:
# knots_list2, coef_list2 = coef2_list_func()

constraints are inconsistent, no solution


In [18]:
# ise2 = ISE_list(knots_list2, coef_list2)
# np.save(".\\weights\\scipy_ise_array", np.array(ise2))

In [19]:
ise2 = np.load(".\\weights\\scipy_ise_array.npy")
min(ise2)

0.0018332057138386783

## [findpeaks Peakdetect](https://erdogant.github.io/findpeaks/pages/html/Peakdetect.html)

In [14]:
def coef3_list_func(n=200):
    coef3_list, knots_list = [], []
    for i in range(n):
        np.random.seed(i)
        x = np.linspace(0, 1, 1000).round(5)
        y = f(x) + np.random.normal(scale=0.3, size=(1000,))
        fp = findpeaks(method="peakdetect", lookahead=100)
        fp_result = fp.fit(y)
        pv_index3 = (fp_result["df"]["peak"]|fp_result["df"]["valley"]).to_numpy()
        pv3 = x[pv_index3]
        knots = get_knot(x, y)
        compre_csr = comprehensive_csr(x=x, y=y, pv_coordinate=pv3[1:-1], pv_index=pv_index3, knots_of_each_part=knots)
        try:
            coef3_list.append(compre_csr.Solve_QP(deg=3, meq=4))
        except ValueError:
            print("constraints are inconsistent, no solution")
            continue
        knots_list.append(compre_csr.comprehensive_knots())
    return knots_list, coef3_list

In [ ]:
# knots_list3, coef_list3 = coef3_list_func()

In [16]:
# ise3 = ISE_list(knots_list3, coef_list3)
# np.save(".\\weights\\fp_ise_array", np.array(ise3))

In [17]:
ise3 = np.load(".\\weights\\fp_ise_array.npy")
min(ise3)

0.001442615492572794